In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import json
import time
from tqdm.notebook import tqdm 
import numpy as np

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [8]:
fips_check = pd.read_csv("../../data/state_data/geo/geocoded/geo_fl.csv") #- original geocoded file
fips_check

,Unnamed: 0,LoanNumber,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,4DigitNAICS,NA,Industry,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,9794577700,491,PPP,FRUIT COVE BAPTIST CHURCH OF JACKSONVILLE FL INC,501 State Road 13,Saint Johns,FL,32259.0,2/19/21,...,8131.0,81.0,Other Services (except Public Administration),NORTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"501 State Road 13, Saint Johns, FL",30.116637,-81.625214,NaN
1,1,1345247301,491,PPP,"ASPIRE HEALTH PARTNERS, INC.","5151 Adanson Street, Suite 200",ORLANDO,FL,32804.0,8/19/21,...,6214.0,62.0,Health Care and Social Assistance,NORTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"5151 Adanson Street, Suite 200, ORLANDO, FL",28.606045,-81.393087,NaN
2,2,1442247205,455,PPP,COLE SCOTT & KISSANE PA,9150 S Dadeland Blvd. Ste 1400,MIAMI,FL,33156.0,7/22/21,...,5411.0,54.0,"Professional, Scientific, and Technical Services",SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"9150 S Dadeland Blvd. Ste 1400, MIAMI, FL",25.684430,-80.314956,NaN
3,3,2030068400,455,PPP,CITRUS WORLD INC.,20205 Hwy 27,Lake Wales,FL,33853.0,10/14/21,...,3114.0,31.0,Manufacturing,SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"20205 Hwy 27, Lake Wales, FL",27.911760,-81.599941,NaN
4,4,2964268209,455,PPP,"LUTHERAN SERVICES FLORIDA, INC",3627 W. WATERS AVE,Tampa,FL,33614.0,9/24/21,...,8131.0,81.0,Other Services (except Public Administration),SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"3627 W. WATERS AVE, Tampa, FL",28.028052,-82.501377,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60036,60036,9950428308,455,PPS,DIGESTIVE & LIVER DISEASE CONSULTANTS OF SOUTH...,7369 Sheridan St,Hollywood,FL,33024.0,11/6/21,...,6211.0,62.0,Health Care and Social Assistance,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"7369 Sheridan St, Hollywood, FL",26.031886,-80.236052,NaN
60037,60037,9963928609,455,PPP,B&B PIZZA PARTNERS LLC,125 N Congress Ave Ste 13-14,Delray Beach,FL,33445.0,NaN,...,7225.0,72.0,Accommodation and Food Services,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"125 N Congress Ave Ste 13-14, Delray Beach, FL",26.464116,-80.093081,NaN
60038,60038,9977848305,455,PPS,BCG FACILITY SERVICES LLC,8608 Little Rd,New Port Richey,FL,34654.0,NaN,...,5617.0,56.0,Administrative and Support and Waste Managemen...,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"8608 Little Rd, New Port Richey, FL",28.282086,-82.675429,NaN
60039,60039,9988227001,455,PPP,"SOUTH TAMPA SWIM SCHOOL, LLC",5016 N COOLIDGE AVE,TAMPA,FL,33614.0,9/30/21,...,6116.0,61.0,Educational Services,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"5016 N COOLIDGE AVE, TAMPA, FL",27.992650,-82.518094,NaN


In [9]:
print(fips_check.isnull().sum())

Unnamed: 0              0
LoanNumber              0
SBAOfficeCode           0
ProcessingMethod        0
BorrowerName            0
                    ...  
job_Range               0
full_add                0
Lat                    13
Long                   13
FIPS_z              60041
Length: 69, dtype: int64


In [10]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [11]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #3

,LoanNumber,full_add,Lat,Long
26535,4394838406,"(813) 731-2210, Lutz, FL",NaN,NaN
42005,6354347105,"7800 NW 25th Street, #1, MIAMI, FL",NaN,NaN
42013,4796727110,"4309 SALISBURY ROAD, JACKSONVILLE, FL",NaN,NaN
42017,9830828806,"1597 The Greens Way, Jacksonville Beach, FL",NaN,NaN
42018,8734157108,"1700 SANS SOUCI BLVD, MIAMI, FL",NaN,NaN
42046,1036188909,"114 S Orange Ave, Orlando, FL",NaN,NaN
42050,1361187209,"3625 Avalon Park W Blvd, ORLANDO, FL",NaN,NaN
42053,5368367009,"1500 NW FEDERAL HWY, STUART, FL",NaN,NaN
42064,7147317207,"6267 OLD WATER OAK RD, TALLAHASSEE, FL",NaN,NaN
42065,3891828508,"1641 N 71st Ter, Hollywood, FL",NaN,NaN


In [12]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #15866

,LoanNumber,full_add,Lat,Long
0,9794577700,"501 State Road 13, Saint Johns, FL",30.116637,-81.625214
1,1345247301,"5151 Adanson Street, Suite 200, ORLANDO, FL",28.606045,-81.393087
2,1442247205,"9150 S Dadeland Blvd. Ste 1400, MIAMI, FL",25.684430,-80.314956
3,2030068400,"20205 Hwy 27, Lake Wales, FL",27.911760,-81.599941
4,2964268209,"3627 W. WATERS AVE, Tampa, FL",28.028052,-82.501377


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../data/state_data/geo/geo_fips/FL_fips_scraped.csv") 
fips_check_scraped.head() 

,Unnamed: 0,LoanNumber,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,4DigitNAICS,NA,Industry,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,9794577700,491,PPP,FRUIT COVE BAPTIST CHURCH OF JACKSONVILLE FL INC,501 State Road 13,Saint Johns,FL,32259.0,2/19/21,...,8131.0,81.0,Other Services (except Public Administration),NORTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"501 State Road 13, Saint Johns, FL",30.116637,-81.625214,1.210902e+14
1,1,1345247301,491,PPP,"ASPIRE HEALTH PARTNERS, INC.","5151 Adanson Street, Suite 200",ORLANDO,FL,32804.0,8/19/21,...,6214.0,62.0,Health Care and Social Assistance,NORTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"5151 Adanson Street, Suite 200, ORLANDO, FL",28.606045,-81.393087,1.209502e+14
2,2,1442247205,455,PPP,COLE SCOTT & KISSANE PA,9150 S Dadeland Blvd. Ste 1400,MIAMI,FL,33156.0,7/22/21,...,5411.0,54.0,"Professional, Scientific, and Technical Services",SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"9150 S Dadeland Blvd. Ste 1400, MIAMI, FL",25.684430,-80.314956,1.208601e+14
3,3,2030068400,455,PPP,CITRUS WORLD INC.,20205 Hwy 27,Lake Wales,FL,33853.0,10/14/21,...,3114.0,31.0,Manufacturing,SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"20205 Hwy 27, Lake Wales, FL",27.911760,-81.599941,1.210501e+14
4,4,2964268209,455,PPP,"LUTHERAN SERVICES FLORIDA, INC",3627 W. WATERS AVE,Tampa,FL,33614.0,9/24/21,...,8131.0,81.0,Other Services (except Public Administration),SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"3627 W. WATERS AVE, Tampa, FL",28.028052,-82.501377,1.205701e+14


In [3]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [4]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [5]:
fips_check_scraped['Lat'].replace('None', np.nan, inplace=True)
fips_check_scraped['Long'].replace('None', np.nan, inplace=True)

In [6]:
#Latitude, Longitude, not null
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna #15866

,LoanNumber,full_add,Lat,Long,FIPS_z
0,9794577700,"501 State Road 13, Saint Johns, FL",30.116637,-81.625214,1.210902e+14
1,1345247301,"5151 Adanson Street, Suite 200, ORLANDO, FL",28.606045,-81.393087,1.209502e+14
2,1442247205,"9150 S Dadeland Blvd. Ste 1400, MIAMI, FL",25.684430,-80.314956,1.208601e+14
3,2030068400,"20205 Hwy 27, Lake Wales, FL",27.911760,-81.599941,1.210501e+14
4,2964268209,"3627 W. WATERS AVE, Tampa, FL",28.028052,-82.501377,1.205701e+14
...,...,...,...,...,...
60036,9950428308,"7369 Sheridan St, Hollywood, FL",26.031886,-80.236052,NaN
60037,9963928609,"125 N Congress Ave Ste 13-14, Delray Beach, FL",26.464116,-80.093081,NaN
60038,9977848305,"8608 Little Rd, New Port Richey, FL",28.282086,-82.675429,NaN
60039,9988227001,"5016 N COOLIDGE AVE, TAMPA, FL",27.992650,-82.518094,NaN


In [7]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #5503

,LoanNumber,full_add,Lat,Long,FIPS_z
4591,2637917101,"660 US Highway One, NORTH PALM BEACH, FL",43.516228,-70.429591,NaN
6167,7500287104,"3311 HIGHWAY 29, CANTONMENT, FL",36.639637,-79.385116,NaN
6839,2398897110,"300 Industrial Park Dr, Perry, FL",32.463852,-83.770625,NaN
9024,4503367106,"VERNIS & BOWLING OF MIAMI PA, MIAMI, FL",41.203322,-77.194525,NaN
9117,7528697002,"2001 N US HWY 27, HOLLYWOOD, FL",32.850258,-84.845482,NaN
...,...,...,...,...,...
60036,9950428308,"7369 Sheridan St, Hollywood, FL",26.031886,-80.236052,NaN
60037,9963928609,"125 N Congress Ave Ste 13-14, Delray Beach, FL",26.464116,-80.093081,NaN
60038,9977848305,"8608 Little Rd, New Port Richey, FL",28.282086,-82.675429,NaN
60039,9988227001,"5016 N COOLIDGE AVE, TAMPA, FL",27.992650,-82.518094,NaN


<h4>FIPS Script

In [8]:
#if geocode file
#lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
#long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [9]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=True)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('12')] #FL
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

The version of chrome cannot be detected. Trying with latest driver version


  0%|          | 0/10471 [00:00<?, ?it/s]

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [13]:
geo_id_list =geo_id 

Length Check

In [14]:
print(len(geo_id_list))

10471


Matching FIPS to DataFrame

In [17]:
#for geocode file
#fips_fill = fips_check_notna.iloc[:7084] #confirm pair to notna DataFrame
#if FIPS available
fips_fill = fips_check_scraped_null.iloc[:10471]

In [18]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
4591,2637917101,"660 US Highway One, NORTH PALM BEACH, FL",43.516228,-70.429591,[]
6167,7500287104,"3311 HIGHWAY 29, CANTONMENT, FL",36.639637,-79.385116,[]
6839,2398897110,"300 Industrial Park Dr, Perry, FL",32.463852,-83.770625,[]
9024,4503367106,"VERNIS & BOWLING OF MIAMI PA, MIAMI, FL",41.203322,-77.194525,[]
9117,7528697002,"2001 N US HWY 27, HOLLYWOOD, FL",32.850258,-84.845482,[]
...,...,...,...,...,...
60036,9950428308,"7369 Sheridan St, Hollywood, FL",26.031886,-80.236052,[120110906012023]
60037,9963928609,"125 N Congress Ave Ste 13-14, Delray Beach, FL",26.464116,-80.093081,[120990069101014]
60038,9977848305,"8608 Little Rd, New Port Richey, FL",28.282086,-82.675429,[121010310082016]
60039,9988227001,"5016 N COOLIDGE AVE, TAMPA, FL",27.992650,-82.518094,[120570026001008]


Remove brackets

In [19]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
4591,2637917101,"660 US Highway One, NORTH PALM BEACH, FL",43.516228,-70.429591,NaN
6167,7500287104,"3311 HIGHWAY 29, CANTONMENT, FL",36.639637,-79.385116,NaN
6839,2398897110,"300 Industrial Park Dr, Perry, FL",32.463852,-83.770625,NaN
9024,4503367106,"VERNIS & BOWLING OF MIAMI PA, MIAMI, FL",41.203322,-77.194525,NaN
9117,7528697002,"2001 N US HWY 27, HOLLYWOOD, FL",32.850258,-84.845482,NaN
...,...,...,...,...,...
60036,9950428308,"7369 Sheridan St, Hollywood, FL",26.031886,-80.236052,120110906012023
60037,9963928609,"125 N Congress Ave Ste 13-14, Delray Beach, FL",26.464116,-80.093081,120990069101014
60038,9977848305,"8608 Little Rd, New Port Richey, FL",28.282086,-82.675429,121010310082016
60039,9988227001,"5016 N COOLIDGE AVE, TAMPA, FL",27.992650,-82.518094,120570026001008


In [20]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #24

,LoanNumber,full_add,Lat,Long,FIPS_z
4591,2637917101,"660 US Highway One, NORTH PALM BEACH, FL",43.516228,-70.429591,NaN
6167,7500287104,"3311 HIGHWAY 29, CANTONMENT, FL",36.639637,-79.385116,NaN
6839,2398897110,"300 Industrial Park Dr, Perry, FL",32.463852,-83.770625,NaN
9024,4503367106,"VERNIS & BOWLING OF MIAMI PA, MIAMI, FL",41.203322,-77.194525,NaN
9117,7528697002,"2001 N US HWY 27, HOLLYWOOD, FL",32.850258,-84.845482,NaN
24053,3778147104,"1645 Florida Line Road, Quincy, FL",30.701605,-84.626124,NaN
28132,4539788907,"31818 US Hwy 19, Palm Beach, FL",30.997660,-84.046845,NaN
33364,9595917006,"21220 US 19 North, CLEARWATER, FL",30.923185,-83.988545,NaN
35540,2514597308,"1006 NORTH US HWY 27, FORT LAUDERDALE, FL",32.850258,-84.845482,NaN
36800,8854147208,"10 Parachute KY Lot 51, HOMESTEAD, FL",37.839333,-84.270018,NaN


Parse for Merge

In [21]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
4591,2637917101,NaN
6167,7500287104,NaN
6839,2398897110,NaN
9024,4503367106,NaN
9117,7528697002,NaN
...,...,...
60036,9950428308,120110906012023
60037,9963928609,120990069101014
60038,9977848305,121010310082016
60039,9988227001,120570026001008


Read in base data

In [22]:
#if from geocode file
#fips_check = pd.read_csv("../../data/state_data/geo/geocoded/geo_fl.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1", "Unnamed: 0.1.1.1.1.1", "Unnamed: 0.1.1.1.1.1.1"], axis =1) 
#fips_check

#if from FIPS file
fips_check = pd.read_csv("../../data/state_data/geo/geo_fips/FL_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0.1.1.1"], axis =1) 
fips_check.head() 


,Unnamed: 0,LoanNumber,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,4DigitNAICS,NA,Industry,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,9794577700,491,PPP,FRUIT COVE BAPTIST CHURCH OF JACKSONVILLE FL INC,501 State Road 13,Saint Johns,FL,32259.0,2/19/21,...,8131.0,81.0,Other Services (except Public Administration),NORTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"501 State Road 13, Saint Johns, FL",30.116637,-81.625214,1.210902e+14
1,1,1345247301,491,PPP,"ASPIRE HEALTH PARTNERS, INC.","5151 Adanson Street, Suite 200",ORLANDO,FL,32804.0,8/19/21,...,6214.0,62.0,Health Care and Social Assistance,NORTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"5151 Adanson Street, Suite 200, ORLANDO, FL",28.606045,-81.393087,1.209502e+14
2,2,1442247205,455,PPP,COLE SCOTT & KISSANE PA,9150 S Dadeland Blvd. Ste 1400,MIAMI,FL,33156.0,7/22/21,...,5411.0,54.0,"Professional, Scientific, and Technical Services",SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"9150 S Dadeland Blvd. Ste 1400, MIAMI, FL",25.684430,-80.314956,1.208601e+14
3,3,2030068400,455,PPP,CITRUS WORLD INC.,20205 Hwy 27,Lake Wales,FL,33853.0,10/14/21,...,3114.0,31.0,Manufacturing,SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"20205 Hwy 27, Lake Wales, FL",27.911760,-81.599941,1.210501e+14
4,4,2964268209,455,PPP,"LUTHERAN SERVICES FLORIDA, INC",3627 W. WATERS AVE,Tampa,FL,33614.0,9/24/21,...,8131.0,81.0,Other Services (except Public Administration),SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"3627 W. WATERS AVE, Tampa, FL",28.028052,-82.501377,1.205701e+14


fips_check_drop = fips_check.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_check_drop
fips_check_drop.to_csv("../../data/state_data/geo/geo_fips/FL_fips_scraped.csv")

Merge with MainFrame

In [23]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,NA,Industry,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,0,9794577700,491,PPP,FRUIT COVE BAPTIST CHURCH OF JACKSONVILLE FL INC,501 State Road 13,Saint Johns,FL,32259.0,2/19/21,...,81.0,Other Services (except Public Administration),NORTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"501 State Road 13, Saint Johns, FL",30.116637,-81.625214,1.210902e+14,NaN
1,1,1345247301,491,PPP,"ASPIRE HEALTH PARTNERS, INC.","5151 Adanson Street, Suite 200",ORLANDO,FL,32804.0,8/19/21,...,62.0,Health Care and Social Assistance,NORTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"5151 Adanson Street, Suite 200, ORLANDO, FL",28.606045,-81.393087,1.209502e+14,NaN
2,2,1442247205,455,PPP,COLE SCOTT & KISSANE PA,9150 S Dadeland Blvd. Ste 1400,MIAMI,FL,33156.0,7/22/21,...,54.0,"Professional, Scientific, and Technical Services",SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"9150 S Dadeland Blvd. Ste 1400, MIAMI, FL",25.684430,-80.314956,1.208601e+14,NaN
3,3,2030068400,455,PPP,CITRUS WORLD INC.,20205 Hwy 27,Lake Wales,FL,33853.0,10/14/21,...,31.0,Manufacturing,SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"20205 Hwy 27, Lake Wales, FL",27.911760,-81.599941,1.210501e+14,NaN
4,4,2964268209,455,PPP,"LUTHERAN SERVICES FLORIDA, INC",3627 W. WATERS AVE,Tampa,FL,33614.0,9/24/21,...,81.0,Other Services (except Public Administration),SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"3627 W. WATERS AVE, Tampa, FL",28.028052,-82.501377,1.205701e+14,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60036,60036,9950428308,455,PPS,DIGESTIVE & LIVER DISEASE CONSULTANTS OF SOUTH...,7369 Sheridan St,Hollywood,FL,33024.0,11/6/21,...,62.0,Health Care and Social Assistance,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"7369 Sheridan St, Hollywood, FL",26.031886,-80.236052,NaN,120110906012023
60037,60037,9963928609,455,PPP,B&B PIZZA PARTNERS LLC,125 N Congress Ave Ste 13-14,Delray Beach,FL,33445.0,NaN,...,72.0,Accommodation and Food Services,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"125 N Congress Ave Ste 13-14, Delray Beach, FL",26.464116,-80.093081,NaN,120990069101014
60038,60038,9977848305,455,PPS,BCG FACILITY SERVICES LLC,8608 Little Rd,New Port Richey,FL,34654.0,NaN,...,56.0,Administrative and Support and Waste Managemen...,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"8608 Little Rd, New Port Richey, FL",28.282086,-82.675429,NaN,121010310082016
60039,60039,9988227001,455,PPP,"SOUTH TAMPA SWIM SCHOOL, LLC",5016 N COOLIDGE AVE,TAMPA,FL,33614.0,9/30/21,...,61.0,Educational Services,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"5016 N COOLIDGE AVE, TAMPA, FL",27.992650,-82.518094,NaN,120570026001008


<b>Fill FIPS Columns - if FIPS file

In [24]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

,Unnamed: 0,LoanNumber,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,Industry,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,0,9794577700,491,PPP,FRUIT COVE BAPTIST CHURCH OF JACKSONVILLE FL INC,501 State Road 13,Saint Johns,FL,32259.0,2/19/21,...,Other Services (except Public Administration),NORTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"501 State Road 13, Saint Johns, FL",30.116637,-81.625214,1.210902e+14,NaN,121090208023015.0
1,1,1345247301,491,PPP,"ASPIRE HEALTH PARTNERS, INC.","5151 Adanson Street, Suite 200",ORLANDO,FL,32804.0,8/19/21,...,Health Care and Social Assistance,NORTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"5151 Adanson Street, Suite 200, ORLANDO, FL",28.606045,-81.393087,1.209502e+14,NaN,120950153001006.0
2,2,1442247205,455,PPP,COLE SCOTT & KISSANE PA,9150 S Dadeland Blvd. Ste 1400,MIAMI,FL,33156.0,7/22/21,...,"Professional, Scientific, and Technical Services",SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"9150 S Dadeland Blvd. Ste 1400, MIAMI, FL",25.684430,-80.314956,1.208601e+14,NaN,120860078082020.0
3,3,2030068400,455,PPP,CITRUS WORLD INC.,20205 Hwy 27,Lake Wales,FL,33853.0,10/14/21,...,Manufacturing,SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"20205 Hwy 27, Lake Wales, FL",27.911760,-81.599941,1.210501e+14,NaN,121050143012025.0
4,4,2964268209,455,PPP,"LUTHERAN SERVICES FLORIDA, INC",3627 W. WATERS AVE,Tampa,FL,33614.0,9/24/21,...,Other Services (except Public Administration),SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"3627 W. WATERS AVE, Tampa, FL",28.028052,-82.501377,1.205701e+14,NaN,120570119082004.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60036,60036,9950428308,455,PPS,DIGESTIVE & LIVER DISEASE CONSULTANTS OF SOUTH...,7369 Sheridan St,Hollywood,FL,33024.0,11/6/21,...,Health Care and Social Assistance,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"7369 Sheridan St, Hollywood, FL",26.031886,-80.236052,NaN,120110906012023,120110906012023
60037,60037,9963928609,455,PPP,B&B PIZZA PARTNERS LLC,125 N Congress Ave Ste 13-14,Delray Beach,FL,33445.0,NaN,...,Accommodation and Food Services,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"125 N Congress Ave Ste 13-14, Delray Beach, FL",26.464116,-80.093081,NaN,120990069101014,120990069101014
60038,60038,9977848305,455,PPS,BCG FACILITY SERVICES LLC,8608 Little Rd,New Port Richey,FL,34654.0,NaN,...,Administrative and Support and Waste Managemen...,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"8608 Little Rd, New Port Richey, FL",28.282086,-82.675429,NaN,121010310082016,121010310082016
60039,60039,9988227001,455,PPP,"SOUTH TAMPA SWIM SCHOOL, LLC",5016 N COOLIDGE AVE,TAMPA,FL,33614.0,9/30/21,...,Educational Services,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"5016 N COOLIDGE AVE, TAMPA, FL",27.992650,-82.518094,NaN,120570026001008,120570026001008


Drop Extra FIPS Columns

In [25]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop

,Unnamed: 0,LoanNumber,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,4DigitNAICS,NA,Industry,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,9794577700,491,PPP,FRUIT COVE BAPTIST CHURCH OF JACKSONVILLE FL INC,501 State Road 13,Saint Johns,FL,32259.0,2/19/21,...,8131.0,81.0,Other Services (except Public Administration),NORTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"501 State Road 13, Saint Johns, FL",30.116637,-81.625214,121090208023015.0
1,1,1345247301,491,PPP,"ASPIRE HEALTH PARTNERS, INC.","5151 Adanson Street, Suite 200",ORLANDO,FL,32804.0,8/19/21,...,6214.0,62.0,Health Care and Social Assistance,NORTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"5151 Adanson Street, Suite 200, ORLANDO, FL",28.606045,-81.393087,120950153001006.0
2,2,1442247205,455,PPP,COLE SCOTT & KISSANE PA,9150 S Dadeland Blvd. Ste 1400,MIAMI,FL,33156.0,7/22/21,...,5411.0,54.0,"Professional, Scientific, and Technical Services",SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"9150 S Dadeland Blvd. Ste 1400, MIAMI, FL",25.684430,-80.314956,120860078082020.0
3,3,2030068400,455,PPP,CITRUS WORLD INC.,20205 Hwy 27,Lake Wales,FL,33853.0,10/14/21,...,3114.0,31.0,Manufacturing,SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"20205 Hwy 27, Lake Wales, FL",27.911760,-81.599941,121050143012025.0
4,4,2964268209,455,PPP,"LUTHERAN SERVICES FLORIDA, INC",3627 W. WATERS AVE,Tampa,FL,33614.0,9/24/21,...,8131.0,81.0,Other Services (except Public Administration),SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"3627 W. WATERS AVE, Tampa, FL",28.028052,-82.501377,120570119082004.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60036,60036,9950428308,455,PPS,DIGESTIVE & LIVER DISEASE CONSULTANTS OF SOUTH...,7369 Sheridan St,Hollywood,FL,33024.0,11/6/21,...,6211.0,62.0,Health Care and Social Assistance,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"7369 Sheridan St, Hollywood, FL",26.031886,-80.236052,120110906012023
60037,60037,9963928609,455,PPP,B&B PIZZA PARTNERS LLC,125 N Congress Ave Ste 13-14,Delray Beach,FL,33445.0,NaN,...,7225.0,72.0,Accommodation and Food Services,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"125 N Congress Ave Ste 13-14, Delray Beach, FL",26.464116,-80.093081,120990069101014
60038,60038,9977848305,455,PPS,BCG FACILITY SERVICES LLC,8608 Little Rd,New Port Richey,FL,34654.0,NaN,...,5617.0,56.0,Administrative and Support and Waste Managemen...,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"8608 Little Rd, New Port Richey, FL",28.282086,-82.675429,121010310082016
60039,60039,9988227001,455,PPP,"SOUTH TAMPA SWIM SCHOOL, LLC",5016 N COOLIDGE AVE,TAMPA,FL,33614.0,9/30/21,...,6116.0,61.0,Educational Services,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"5016 N COOLIDGE AVE, TAMPA, FL",27.992650,-82.518094,120570026001008


In [27]:
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop.loc[fips_merge["FIPS_z"].isnull()] #37337 >30260 > 25328 > 10484

,LoanNumber,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,LoanStatus,...,4DigitNAICS,NA,Industry,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
4591,2637917101,455,PPP,"CORNERSTONE DENTISTRY HOLDINGS, PLLC",660 US Highway One,NORTH PALM BEACH,FL,33408.0,4/21/21,Paid in Full,...,6212.0,62.0,Health Care and Social Assistance,SOUTH FLORIDA DISTRICT OFFICE,c $1-2 million,50 to 99,"660 US Highway One, NORTH PALM BEACH, FL",43.516228,-70.429591,NaN
6167,7500287104,491,PPP,"BUTLER FOODS OF PENSACOLA, INC.",3311 HIGHWAY 29,CANTONMENT,FL,32533.0,4/6/21,Paid in Full,...,4244.0,42.0,Wholesale Trade,NORTH FLORIDA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"3311 HIGHWAY 29, CANTONMENT, FL",36.639637,-79.385116,NaN
6839,2398897110,491,PPP,RDS MANUFACTURING INC,300 Industrial Park Dr,Perry,FL,32348.0,5/25/21,Paid in Full,...,3324.0,33.0,Manufacturing,NORTH FLORIDA DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"300 Industrial Park Dr, Perry, FL",32.463852,-83.770625,NaN
9024,4503367106,455,PPP,VERNIS & BOWLING OF MIAMI P.A.,VERNIS & BOWLING OF MIAMI PA,MIAMI,FL,33181.0,2/20/21,Paid in Full,...,5411.0,54.0,"Professional, Scientific, and Technical Services",SOUTH FLORIDA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"VERNIS & BOWLING OF MIAMI PA, MIAMI, FL",41.203322,-77.194525,NaN
9117,7528697002,455,PPP,TWS FABRICATORS INC.,2001 N US HWY 27,HOLLYWOOD,FL,33029.0,1/21/21,Paid in Full,...,2381.0,23.0,Construction,SOUTH FLORIDA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"2001 N US HWY 27, HOLLYWOOD, FL",32.850258,-84.845482,NaN
24053,3778147104,491,PPP,"S.K. ENTERPRISES OF NORTH FLORIDA, INC.",1645 Florida Line Road,Quincy,FL,32351.0,7/29/21,Paid in Full,...,4244.0,42.0,Wholesale Trade,NORTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1645 Florida Line Road, Quincy, FL",30.701605,-84.626124,NaN
26535,4394838406,455,PPS,TRAVELIN LIGHT INC,(813) 731-2210,Lutz,FL,33559.0,NaN,Exemption 4,...,7225.0,72.0,Accommodation and Food Services,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"(813) 731-2210, Lutz, FL",NaN,NaN,NaN
28132,4539788907,455,PPS,PROMISE PHARMACY LLC,31818 US Hwy 19,Palm Beach,FL,34684.0,NaN,Exemption 4,...,4461.0,44.0,Retail Trade,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"31818 US Hwy 19, Palm Beach, FL",30.997660,-84.046845,NaN
33364,9595917006,455,PPP,"DANJI, INC.",21220 US 19 North,CLEARWATER,FL,33765.0,2/9/21,Paid in Full,...,7225.0,72.0,Accommodation and Food Services,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"21220 US 19 North, CLEARWATER, FL",30.923185,-83.988545,NaN
35540,2514597308,455,PPP,"ADVENTURE PARK MANAGEMENT, INC.",1006 NORTH US HWY 27,FORT LAUDERDALE,FL,33327.0,9/23/21,Paid in Full,...,5615.0,56.0,Administrative and Support and Waste Managemen...,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1006 NORTH US HWY 27, FORT LAUDERDALE, FL",32.850258,-84.845482,NaN


Convert to File

In [28]:
fips_FL = fips_merge_drop
#fips_TN = fips_merge_drop

In [29]:
fips_FL.to_csv("../../data/state_data/geo/geo_fips/FL_fips_scraped.csv") 

Review & Compare

In [2]:
fips_FL = pd.read_csv("../../data/state_data/geo/geo_fips/FL_fips_scraped.csv") 
fips_FL

,Unnamed: 0,LoanNumber,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,4DigitNAICS,NA,Industry,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,9794577700,491,PPP,FRUIT COVE BAPTIST CHURCH OF JACKSONVILLE FL INC,501 State Road 13,Saint Johns,FL,32259.0,2/19/21,...,8131.0,81.0,Other Services (except Public Administration),NORTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"501 State Road 13, Saint Johns, FL",30.116637,-81.625214,1.210902e+14
1,1,1345247301,491,PPP,"ASPIRE HEALTH PARTNERS, INC.","5151 Adanson Street, Suite 200",ORLANDO,FL,32804.0,8/19/21,...,6214.0,62.0,Health Care and Social Assistance,NORTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"5151 Adanson Street, Suite 200, ORLANDO, FL",28.606045,-81.393087,1.209502e+14
2,2,1442247205,455,PPP,COLE SCOTT & KISSANE PA,9150 S Dadeland Blvd. Ste 1400,MIAMI,FL,33156.0,7/22/21,...,5411.0,54.0,"Professional, Scientific, and Technical Services",SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"9150 S Dadeland Blvd. Ste 1400, MIAMI, FL",25.684430,-80.314956,1.208601e+14
3,3,2030068400,455,PPP,CITRUS WORLD INC.,20205 Hwy 27,Lake Wales,FL,33853.0,10/14/21,...,3114.0,31.0,Manufacturing,SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"20205 Hwy 27, Lake Wales, FL",27.911760,-81.599941,1.210501e+14
4,4,2964268209,455,PPP,"LUTHERAN SERVICES FLORIDA, INC",3627 W. WATERS AVE,Tampa,FL,33614.0,9/24/21,...,8131.0,81.0,Other Services (except Public Administration),SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"3627 W. WATERS AVE, Tampa, FL",28.028052,-82.501377,1.205701e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60036,60036,9950428308,455,PPS,DIGESTIVE & LIVER DISEASE CONSULTANTS OF SOUTH...,7369 Sheridan St,Hollywood,FL,33024.0,11/6/21,...,6211.0,62.0,Health Care and Social Assistance,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"7369 Sheridan St, Hollywood, FL",26.031886,-80.236052,1.201109e+14
60037,60037,9963928609,455,PPP,B&B PIZZA PARTNERS LLC,125 N Congress Ave Ste 13-14,Delray Beach,FL,33445.0,NaN,...,7225.0,72.0,Accommodation and Food Services,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"125 N Congress Ave Ste 13-14, Delray Beach, FL",26.464116,-80.093081,1.209901e+14
60038,60038,9977848305,455,PPS,BCG FACILITY SERVICES LLC,8608 Little Rd,New Port Richey,FL,34654.0,NaN,...,5617.0,56.0,Administrative and Support and Waste Managemen...,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"8608 Little Rd, New Port Richey, FL",28.282086,-82.675429,1.210103e+14
60039,60039,9988227001,455,PPP,"SOUTH TAMPA SWIM SCHOOL, LLC",5016 N COOLIDGE AVE,TAMPA,FL,33614.0,9/30/21,...,6116.0,61.0,Educational Services,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"5016 N COOLIDGE AVE, TAMPA, FL",27.992650,-82.518094,1.205700e+14


In [3]:
fips_FL.loc[fips_FL["FIPS_z"].isnull()].count() #37337 >30260 > 25328

Unnamed: 0          26
LoanNumber          26
SBAOfficeCode       26
ProcessingMethod    26
BorrowerName        26
                    ..
job_Range           26
full_add            26
Lat                 13
Long                13
FIPS_z               0
Length: 69, dtype: int64

In [4]:
fips_FL.shape[0]
print(fips_FL.isnull().sum()) #check after FL process 
# note: FL !!

Unnamed: 0           0
LoanNumber           0
SBAOfficeCode        0
ProcessingMethod     0
BorrowerName         0
                    ..
job_Range            0
full_add             0
Lat                 13
Long                13
FIPS_z              26
Length: 69, dtype: int64


In [5]:
#3635048304, 9551438401, 6775747900
fips_FL.loc[fips_FL["LoanNumber"]== 3635048304]

,Unnamed: 0,LoanNumber,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,4DigitNAICS,NA,Industry,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
23529,23529,3635048304,491,PPS,UNITED ELECTRICAL CONTRACTORS INC,744 S Rossiter St,Mount Dora,FL,32757.0,9/14/21,...,2382.0,23.0,Construction,NORTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"744 S Rossiter St, Mount Dora, FL",28.790232,-81.633567,1.206903e+14


In [6]:
fips_FL.loc[fips_FL["LoanNumber"]== 9551438401]

,Unnamed: 0,LoanNumber,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,4DigitNAICS,NA,Industry,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
23530,23530,9551438401,455,PPS,OSPREY LEISURE LLC,1660 S Tamiami Trl Mail Rt 3.12.18,Osprey,FL,34229.0,NaN,...,7211.0,72.0,Accommodation and Food Services,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1660 S Tamiami Trl Mail Rt 3.12.18, Osprey, FL",27.172464,-82.479732,1.211500e+14


In [7]:
fips_FL.loc[fips_FL["LoanNumber"]== 6775747900]

,Unnamed: 0,LoanNumber,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,4DigitNAICS,NA,Industry,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
38683,38683,6775747900,455,PPP,"TEDDY PIERRE-LOUIS, PA, LLC",282 Northeast 166th Street,Miami,FL,33162.0,9/23/21,...,5312.0,53.0,Real Estate and Rental and Leasing,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"282 Northeast 166th Street, Miami, FL",25.927417,-80.194678,1.208600e+14
